In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import linalg

In [2]:
def SU2(r):   # r is the distance from I
    x = np.random.uniform(-1,1,4)  
    x0 =np.sqrt(x[1]**2+x[2]**2+x[3]**2)
    x[1:] = x[1:]/x0*r
    x[0] = np.sqrt(1-r**2) # in this case norm(x)=1
    z1 = complex(x[1],x[2])
    z2 = complex(-x[1],x[2])
    z0 = complex(x[0],x[3])
    z4 = complex(x[0],-x[3])
    M = np.matrix([[z0,z1],[z2,z4]])
    return M

In [3]:
def reSU2(Matrix):
    a = Matrix[0,0]
    b = Matrix[0,1]
    n = a*a.conjugate()+b*b.conjugate()
    n0 = np.sqrt(n)
    a = a/n0
    b = b/n0
    Matrix = np.matrix([[a,b],[-b.conjugate(),a.conjugate()]])
    return Matrix

In [4]:
#generate Link variables in the lattice, with the size (T,X,Y,Z)
#the link only defined in the positive direction,also restore the complex conjugate for the inverse
def GenerateLink(T,X,Y,Z):
    return np.zeros((T,X,Y,Z,8,2,2),dtype = np.complex128)

In [5]:
# initial the Link as a cold start
def coldIni(Link):
    Sha = Link.shape
    for t in range(Sha[0]):
        for x in range(Sha[1]):
            for y in range(Sha[2]):
                for z in range(Sha[3]):
                    for k in range(4):
                        Link[t,x,y,z,k,:,:] = np.identity(2,dtype=np.complex128)
                        Link[t,x,y,z,k+4,:,:] = np.identity(2,dtype=np.complex128)
        print('\r',(t+1)/Sha[0],end='')

In [6]:
def hotIni(Link):
    Sha = Link.shape
    for t in range(Sha[0]):
        for x in range(Sha[1]):
            for y in range(Sha[2]):
                for z in range(Sha[3]):
                    for k in range(4):
                        M1 = SU2()
                        M2 = M1.getH()
                        Link[t,x,y,z,k,:,:] = M1
                        Link[t,x,y,z,k+4,:,:] = M2
        print('/r',t/Sha[0],end='')

In [7]:
#  bd condition
def Move(u,a,t,x,y,z,sh): # a single move on lattice
    if u == 0:
        t = (t+a)%sh[0]
    if u == 1:
        x = (x+a)%sh[1]
    if u == 2:
        y = (y+a)%sh[2]
    if u == 3:
        z = (z+a)%sh[3]
    return t,x,y,z



In [8]:
def update(Link,u,t,x,y,z,beta):  # update one point and one direction
    sh = Link.shape
    A = np.zeros((2,2),dtype = np.complex128)
    for v in range(4):
        if v != u:
            t1,x1,y1,z1 = t,x,y,z
            t2,x2,y2,z2 = t,x,y,z
            t1,x1,y1,z1 = Move(u,1,t1,x1,y1,z1,sh)   
            Uv1 = Link[t1,x1,y1,z1,v,:,:] 
            t1,x1,y1,z1 = Move(u,-1,t1,x1,y1,z1,sh) 
            t1,x1,y1,z1 = Move(v,+1,t1,x1,y1,z1,sh)            
            Uuc = Link[t1,x1,y1,z1,u+4,:,:] 
            t1,x1,y1,z1 = Move(v,-1,t1,x1,y1,z1,sh) 
            Uvc = Link[t1,x1,y1,z1,v+4,:,:]      
            t2,x2,y2,z2 = Move(u,1,t2,x2,y2,z2,sh)   
            t2,x2,y2,z2 = Move(v,-1,t2,x2,y2,z2,sh)
            Uv2c = Link[t2,x2,y2,z2,v+4,:,:]           
            t2,x2,y2,z2 = Move(u,-1,t2,x2,y2,z2,sh)
            Uu2c = Link[t2,x2,y2,z2,u+4,:,:]
            Uv2  = Link[t2,x2,y2,z2,v,:,:]           
            A = A + Uv1@Uuc@Uvc + Uv2c@Uu2c@Uv2
    acceptence = 0
    hits = 10  #update each point several(10) times before move to another point    
    for i in range(hits):        
            newLink = SU2(0.45)@Link[t,x,y,z,u,:,:] #0.45 is choosen to make acceptence rate around 0.5
            Tr = np.trace((newLink - Link[t,x,y,z,u,:,:])@A)
            deltaS = -(beta/2)*Tr.real
            if  np.exp(-deltaS) > np.random.random(): 
                renew = reSU2(newLink)
                Link[t,x,y,z,u,:,:] = renew
                Link[t,x,y,z,u+4,:,:] = np.matrix(renew).getH()
                acceptence = acceptence +1
    return Link,acceptence/hits

In [9]:
'''
T = 32
X = 12
Y = 12
Z = 12
beta = 2.3
Link = GenerateLink(T,X,Y,Z)
coldIni(Link)
'''

'\nT = 32\nX = 12\nY = 12\nZ = 12\nbeta = 2.3\nLink = GenerateLink(T,X,Y,Z)\ncoldIni(Link)\n'

In [10]:
'''
acc = []
# 29/02/2020  around 1.00pm first start 0-200 config saved 
# end at 20:11
for N in range(200):
    acceptence = np.zeros((T,X,Y,Z,4))
    for t in range(T):
        for x in range(X):
            for y in range(Y):
                for z in range(Z):
                    for k in range(4):
                        Link,acceptence[t,x,y,z,k] = update(Link,k,t,x,y,z,beta)
    acc.append(np.mean(acceptence))
    filename = 'F:\QCD\gauge_configs\\'+ str(N)
    np.save(filename,Link)
    print(N)
'''

"\nacc = []\n# 29/02/2020  around 1.00pm first start 0-200 config saved \n# end at 20:11\nfor N in range(200):\n    acceptence = np.zeros((T,X,Y,Z,4))\n    for t in range(T):\n        for x in range(X):\n            for y in range(Y):\n                for z in range(Z):\n                    for k in range(4):\n                        Link,acceptence[t,x,y,z,k] = update(Link,k,t,x,y,z,beta)\n    acc.append(np.mean(acceptence))\n    filename = 'F:\\QCD\\gauge_configs\\'+ str(N)\n    np.save(filename,Link)\n    print(N)\n"

In [11]:
#np.mean(acc)

In [12]:
#np.var(acc)

acceptence rate: 0.497867(4)

In [9]:
T = 32
X = 12
Y = 12
Z = 12
beta = 2.3
acc = []
# 01/03/2020  13:00  200-399
# 02/03/2020  13:05  400-599  17:40 102 (4*60+30)/100=2.7 min with 1 config  19:52 150
Link = np.load('F:\QCD\gauge_configs\\\899.npy')  # one should change this when programme start
for N in range(200):
    acceptence = np.zeros((T,X,Y,Z,4))
    for t in range(T):
        for x in range(X):
            for y in range(Y):
                for z in range(Z):
                    for k in range(4):
                        Link,acceptence[t,x,y,z,k] = update(Link,k,t,x,y,z,beta)
    acc.append(np.mean(acceptence))
    filename = 'F:\QCD\gauge_configs\\'+ str(N+900) # also change this one
    np.save(filename,Link)
    print('\r',N,end='')

 199